In [1]:
# !pip -q install transformers datasets peft accelerate adapter-transformers bitsandbytes

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 

In [1]:
import sys
import subprocess

def in_venv():
    return sys.prefix != sys.base_prefix

in_venv()

True

In [2]:
!pip install peft --no-dependencies

Defaulting to user installation because normal site-packages is not writeable


In [3]:
# !huggingface-cli login
from huggingface_hub import login
login()

In [4]:
from datasets import load_dataset
dataset = load_dataset("boolq")

Using custom data configuration boolq-150dc58bba7bf36d
Found cached dataset parquet (/u/poyuchen/.cache/huggingface/datasets/parquet/boolq-150dc58bba7bf36d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
def format_example(example):
    question = example['question']
    passage = example['passage']
    answer = 'Yes' if example['answer'] else 'No'
    prompt = f"Question: {question}\nPassage: {passage}\nAnswer:"
    return {'input_text': prompt, 'target_text': answer}
formatted_dataset = dataset.map(format_example, remove_columns=dataset['train'].column_names)

Loading cached processed dataset at /u/poyuchen/.cache/huggingface/datasets/parquet/boolq-150dc58bba7bf36d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-a1e8f8b774871ade.arrow
Loading cached processed dataset at /u/poyuchen/.cache/huggingface/datasets/parquet/boolq-150dc58bba7bf36d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-d9531fa37afdd21a.arrow


In [6]:
from transformers import AutoTokenizer

# tokenizer = LlamaTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf')
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    inputs = [f"{input_text} {target_text}" for input_text, target_text in zip(examples['input_text'], examples['target_text'])]

    # Tokenize the combined text and return PyTorch tensors
    model_inputs = tokenizer(
        inputs,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_tensors='pt',  # Ensure tensors are returned
    )

    # Create labels by cloning the input IDs
    labels = model_inputs['input_ids'].clone()

    # Determine the length of the input_text for each example
    input_lengths = []
    for input_text in examples['input_text']:
        input_ids = tokenizer(
            input_text,
            max_length=512,
            truncation=True,
            add_special_tokens=False,  # Do not add special tokens here
        )['input_ids']
        input_lengths.append(len(input_ids))

    # Mask the labels for the input_text portion
    for i, input_length in enumerate(input_lengths):
        labels[i, :input_length] = -100  # Use -100 to ignore the input tokens

    model_inputs['labels'] = labels

    return model_inputs

tokenized_dataset = formatted_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=formatted_dataset['train'].column_names,
)

Loading cached processed dataset at /u/poyuchen/.cache/huggingface/datasets/parquet/boolq-150dc58bba7bf36d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-b3d5c1713fbdc4df.arrow
Loading cached processed dataset at /u/poyuchen/.cache/huggingface/datasets/parquet/boolq-150dc58bba7bf36d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-dd5decfda74ec72d.arrow


In [7]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # We are not using masked language modeling
)

In [8]:
# model = LlamaForCausalLM.from_pretrained('meta-llama/Llama-2-7b-hf', load_in_8bit=True, device_map='auto')
from peft import PromptTuningConfig, get_peft_model, TaskType
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")  

prompt_config = PromptTuningConfig(
    num_virtual_tokens=20,  # Number of virtual prompt tokens
    task_type='CAUSAL_LM'   # Task type for causal language modeling
)

model = get_peft_model(model, prompt_config)

In [9]:
from transformers import TrainingArguments

# Set the pad_token to the eos_token
tokenizer.pad_token = tokenizer.eos_token

# Update model's config
model.config.pad_token_id = tokenizer.pad_token_id
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=100,
    # evaluation_strategy='steps',
    # eval_steps=1000,
    save_steps=3000,
    logging_dir='./logs',
    logging_steps=1000,
    learning_rate=1e-4,
    report_to='none',
)
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [11]:
trainer.train()

Step,Training Loss
1000,1.970400
2000,1.963700
3000,1.966800
4000,1.947900
5000,1.961100
6000,1.956900
7000,1.948700
8000,1.955700
9000,1.951500
10000,1.959100


TrainOutput(global_step=23570, training_loss=1.950791323260832, metrics={'train_runtime': 7877.085, 'train_samples_per_second': 11.968, 'train_steps_per_second': 2.992, 'total_flos': 2.8182062278115328e+17, 'train_loss': 1.950791323260832, 'epoch': 10.0})

In [12]:
from datasets import load_dataset

# Load the validation split
dataset = load_dataset("boolq", split='validation')
formatted_dataset = dataset.map(format_example)

Using custom data configuration boolq-150dc58bba7bf36d
Found cached dataset parquet (/u/poyuchen/.cache/huggingface/datasets/parquet/boolq-150dc58bba7bf36d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Loading cached processed dataset at /u/poyuchen/.cache/huggingface/datasets/parquet/boolq-150dc58bba7bf36d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-1915d7e2291719be.arrow


In [13]:
import torch
model.eval()

def generate_answer(example):
    input_text = example['input_text']

    # Tokenize and move inputs to the model's device
    inputs = tokenizer(
        input_text,
        return_tensors='pt',
        max_length=512,
        truncation=True,
        padding='max_length',
    )
    input_ids = inputs['input_ids'].to(model.device)
    attention_mask = inputs['attention_mask'].to(model.device)

    # Generate the model's output
    with torch.no_grad():
        output_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=15,
            do_sample=False,
            num_beams=1,
            pad_token_id=tokenizer.eos_token_id,
        )

    # Decode the generated tokens
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Extract the answer by removing the input prompt
    answer = generated_text[len(input_text):].strip()

    # Handle empty outputs
    if not answer:
        answer = "No"
        return {'predicted_answer': answer}

    # # Keep only the first word (Yes or No)
    # answer = answer.split()[0]
#     print(answer)

    # # Handle unexpected outputs
    if 'yes' in answer.lower():
        answer = 'Yes'
    elif 'no' in answer.lower():
        answer = 'No'
    else:
        answer = 'No'

    return {'predicted_answer': answer}


# Map the generate_answer function over the validation set
predictions = formatted_dataset.map(generate_answer, batched=False)


  0%|          | 0/3270 [00:00<?, ?ex/s]

/w/284/poyuchen/MAT1510_Final/venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/w/284/poyuchen/MAT1510_Final/venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/w/284/poyuchen/MAT1510_Final/venv/lib/python3.10/site-packages/peft/peft_model.py:1755: UserWarning: Position ids are not supported for parameter efficient tuning. Ignoring position ids.
  warnings.warn("Position ids are not supported for parameter efficient tuning. Ignoring position ids.")
Starting from v4.46, the `logits` model output will hav

In [14]:
import numpy as np

def compute_accuracy(predictions):
    # Convert the true and predicted answers to lowercase for consistency
    true_answers = [answer.lower() for answer in predictions['target_text']]
    predicted_answers = [answer.lower() for answer in predictions['predicted_answer']]

    # Calculate the number of correct predictions
    correct_predictions = sum([
        true == pred for true, pred in zip(true_answers, predicted_answers)
    ])

    # Compute accuracy
    accuracy = correct_predictions / len(true_answers)
    return accuracy

# Calculate accuracy
accuracy = compute_accuracy(predictions)
print(f"Accuracy on the validation set: {accuracy * 100:.2f}%")

Accuracy on the validation set: 41.10%
